# 샤크 리그 점수

## 파이썬 라이브러리 임포트

In [1]:
import pandas as pd
import logging
import swc_simple_client as swc

## 로깅 설정

In [2]:
for handler in logging.root.handlers[:]:
    logging.root.removeHandler(handler)

logging.basicConfig(
    filename='shark_notebook.log',  
    level=logging.INFO,  
)

## 노트북 변수 설정

In [3]:
base_url = "https://ominous-space-trout-5v46p9ppv4r3vw-8000.app.github.dev/"


## 최대 득점 조회
### 엔드포인트: LIST_WEEKS_ENDPOINT 

In [4]:
week_api_response = swc.call_api_endpoint(base_url,swc.LIST_WEEKS_ENDPOINT)
weeks_df = pd.DataFrame(week_api_response.json())
weeks_df['year'] = weeks_df['week_number'].str.slice(0, 4).astype(int)
weeks_df['week'] = weeks_df['week_number'].str.slice(4, 6).astype(int)

weeks_df = weeks_df.query('week <= 14')

max_totals_grouped_df = weeks_df.groupby('year').agg(
    ppr_12_max_points=('ppr_12_max_points', 'sum'), 
    half_ppr_8_max_points=('half_ppr_8_max_points', 'sum'))

display(max_totals_grouped_df)

,ppr_12_max_points,half_ppr_8_max_points
year,,
2023,16716.0,12304.0


In [5]:
#tabulate로 출력

from tabulate import tabulate

# 출력용 복사본 생성
max_totals_grouped_df_copy = max_totals_grouped_df.copy().reset_index()

# 천 단위 콤마 적용
max_totals_grouped_df_copy['ppr_12_max_points'] = max_totals_grouped_df_copy['ppr_12_max_points'].map('{:,.0f}'.format)
max_totals_grouped_df_copy['half_ppr_8_max_points'] = max_totals_grouped_df_copy['half_ppr_8_max_points'].map('{:,.0f}'.format)

max_totals_grouped_df_cols = ['시즌', 'PPR(12팀) 최대 득점 합계', 'Half-PPR(8팀) 최대 득점 합계']

# tabulate 출력
print(
    tabulate(
        max_totals_grouped_df_copy,
        headers=max_totals_grouped_df_cols,
        tablefmt='github',
        showindex=False,
        colalign=("right", "right", "right")
    )
)

|   시즌 |   PPR(12팀) 최대 득점 합계 |   Half-PPR(8팀) 최대 득점 합계 |
|--------|----------------------------|--------------------------------|
|   2023 |                     16,716 |                         12,304 |


## 리그 점수 산출 방식 조회
### 엔드포인트: LIST_LEAGUES_ENDPOINT

In [6]:
league_api_response = swc.call_api_endpoint(base_url,swc.LIST_LEAGUES_ENDPOINT)
leagues_df = pd.DataFrame(league_api_response.json())

# 불필요 컬럼 제거
leagues_df = leagues_df.drop(columns=['teams','last_changed_date'])
display(leagues_df)

,league_id,league_name,scoring_type,league_size
0,5001,Pigskin Prodigal Fantasy League,PPR,12
1,5002,Recurring Champions League,Half-PPR,8
2,5003,AHAHFZZFFFL,Half-PPR,8
3,5004,Gridiron Gurus Fantasy League,PPR,12
4,5005,Best League Ever,PPR,12


In [7]:
leagues_df_copy = leagues_df.copy()

leagues_df_copy_cols = ['리그 ID', '리그명', '점수 산출 방식', '팀 수']

# tabulate 출력
print(
    tabulate(
        leagues_df_copy,
        headers=leagues_df_copy_cols,
        tablefmt='github',
        showindex=False,
        colalign=("right", "left", "center", "right")
    )
)


|   리그 ID | 리그명                          |  점수 산출 방식  |   팀 수 |
|-----------|---------------------------------|------------------|---------|
|      5001 | Pigskin Prodigal Fantasy League |       PPR        |      12 |
|      5002 | Recurring Champions League      |     Half-PPR     |       8 |
|      5003 | AHAHFZZFFFL                     |     Half-PPR     |       8 |
|      5004 | Gridiron Gurus Fantasy League   |       PPR        |      12 |
|      5005 | Best League Ever                |       PPR        |      12 |


## 정규 시즌 총 득점 조회 - 팀별
### 엔드포인트: LIST_TEAMS_ENDPOINT

In [8]:
team_api_response = swc.call_api_endpoint(base_url,swc.LIST_TEAMS_ENDPOINT)

teams_parsed_df = pd.json_normalize(team_api_response.json(), 'weekly_scores', 
                                    ['league_id', 'team_id', 'team_name'])

teams_parsed_df['year'] = (teams_parsed_df['week_number']
                           .str.slice(0, 4).astype(int))
teams_parsed_df['week'] = (teams_parsed_df['week_number']
                           .str.slice(4, 6).astype(int))

#정규 시즌 팀만 조회
teams_regular_season_df = teams_parsed_df.query('week <= 14')

#팀 시즌 총 득점 조회
team_totals_df = teams_regular_season_df.groupby(
    ['league_id', 'team_id', 'year'], as_index = False
    )['fantasy_points'].sum()

team_totals_df.head()

,league_id,team_id,year,fantasy_points
0,5001,1001,2023,1238.0
1,5001,1002,2023,1216.0
2,5001,1003,2023,1193.0
3,5001,1004,2023,1071.0
4,5001,1005,2023,958.0


In [9]:
team_totals_df_copy = team_totals_df.copy()

team_totals_df_copy_cols = ['리그 ID', '팀 ID', '시즌', '시즌 총 득점']

# 천 단위 콤마 적용
team_totals_df_copy['fantasy_points'] = team_totals_df_copy['fantasy_points'].map('{:,.0f}'.format)

# tabulate 출력
print(
    tabulate(
        team_totals_df_copy.head(10),
        headers=team_totals_df_copy_cols,
        tablefmt='github',
        showindex=False,
        colalign=("right", "right", "right", "right")
    )
)


|   리그 ID |   팀 ID |   시즌 |   시즌 총 득점 |
|-----------|---------|--------|----------------|
|      5001 |    1001 |   2023 |          1,238 |
|      5001 |    1002 |   2023 |          1,216 |
|      5001 |    1003 |   2023 |          1,193 |
|      5001 |    1004 |   2023 |          1,071 |
|      5001 |    1005 |   2023 |            958 |
|      5001 |    1006 |   2023 |          1,319 |
|      5001 |    1007 |   2023 |          1,048 |
|      5001 |    1008 |   2023 |          1,276 |
|      5001 |    1009 |   2023 |          1,157 |
|      5001 |    1010 |   2023 |            975 |


## 리그 균형 점수
### 정규 시즌 중 득점의 변동계수 사용

In [10]:
league_stats_df = team_totals_df.groupby(['league_id','year']).agg(
    league_points_sum=('fantasy_points', 'sum'),
    league_points_mean=('fantasy_points', 'mean'),
    league_points_stdev=('fantasy_points', 'std'),
    league_balance_score=('fantasy_points', 
                          lambda x: (100 -(x.std() / x.mean()) * 100))
).reset_index()

display(league_stats_df.sort_values(by='league_balance_score', ascending=False))

,league_id,year,league_points_sum,league_points_mean,league_points_stdev,league_balance_score
1,5002,2023,9126.0,1140.750000,58.801725,94.845345
0,5001,2023,14052.0,1171.000000,128.872735,88.994643
3,5004,2023,13752.0,1146.000000,148.203177,87.067786
4,5005,2023,13313.0,1109.416667,145.786555,86.859170
2,5003,2023,8599.0,1074.875000,157.835753,85.315897


In [11]:
league_stats_df_copy = league_stats_df.sort_values(by='league_balance_score', ascending=False)

# 천 단위 콤마 적용
league_stats_df_copy['league_points_sum'] = league_stats_df_copy['league_points_sum'].map('{:,.1f}'.format)
league_stats_df_copy['league_points_mean'] = league_stats_df_copy['league_points_mean'].map('{:,.4f}'.format)
league_stats_df_copy['league_points_stdev'] = league_stats_df_copy['league_points_stdev'].map('{:,.3f}'.format)
league_stats_df_copy['league_balance_score'] = league_stats_df_copy['league_balance_score'].map('{:,.4f}'.format)

league_stats_df_copy_cols = ['리그 ID', '시즌', 
                             '리그 전체 누적 득점 합계', 
                             '팀당 평균 득점', 
                             '팀 득점 표준편차', 
                             '리그 균형도 점수']

# tabulate 출력
print(
    tabulate(
        league_stats_df_copy.head(10),
        headers=league_stats_df_copy_cols,
        tablefmt='fancy',
        showindex=False,
        colalign=("right", "right", "right", "right", "right", "right")
    )
)


  리그 ID    시즌    리그 전체 누적 득점 합계    팀당 평균 득점    팀 득점 표준편차    리그 균형도 점수
---------  ------  --------------------------  ----------------  ------------------  ------------------
     5002    2023                     9,126.0        1,140.7500              58.802             94.8453
     5001    2023                    14,052.0        1,171.0000             128.873             88.9946
     5004    2023                    13,752.0        1,146.0000             148.203             87.0678
     5005    2023                    13,313.0        1,109.4167             145.787             86.8592
     5003    2023                     8,599.0        1,074.8750             157.836             85.3159


## 라인업 효율성 점수
### 실제 득점과 최대 가능 득점 비교

In [12]:
league_stats_with_league_max_df = (league_stats_df[
    ['league_id','year', 'league_points_sum','league_balance_score']]
               .merge(max_totals_grouped_df,left_on = 'year', right_on='year'))

combined_metrics_df = (leagues_df[
    ['league_id','league_name','scoring_type', 'league_size']]
    .merge(league_stats_with_league_max_df, 
           left_on = 'league_id', right_on = 'league_id'))

combined_metrics_df['league_juice_score'] = combined_metrics_df.apply(
    lambda row: (
        100 * (row['league_points_sum'] / row['ppr_12_max_points'])
        if (row['scoring_type'] == 'PPR' and row['league_size'] == 12) 
        else (
            100 * (row['league_points_sum'] / row['half_ppr_8_max_points'])
            if (row['scoring_type'] == 'Half-PPR' and row['league_size'] == 8) 
            else None
        )
    ),
    axis=1
)

combined_metrics_df = (combined_metrics_df.drop(
    columns=['scoring_type','league_size','league_points_sum'
             ,'ppr_12_max_points','half_ppr_8_max_points',] )
)
display(combined_metrics_df)

,league_id,league_name,year,league_balance_score,league_juice_score
0,5001,Pigskin Prodigal Fantasy League,2023,88.994643,84.063173
1,5002,Recurring Champions League,2023,94.845345,74.171001
2,5003,AHAHFZZFFFL,2023,85.315897,69.887841
3,5004,Gridiron Gurus Fantasy League,2023,87.067786,82.268485
4,5005,Best League Ever,2023,86.859170,79.642259


In [13]:
combined_metrics_df_copy = combined_metrics_df.copy()

# 천 단위 콤마 적용
combined_metrics_df_copy['league_balance_score'] = combined_metrics_df_copy['league_balance_score'].map('{:,.4f}'.format)
combined_metrics_df_copy['league_juice_score'] = combined_metrics_df_copy['league_juice_score'].map('{:,.3f}'.format)

combined_metrics_df_copy_cols = [
    '리그 ID',
    '리그 이름',
    '시즌',
    '리그 균형도 점수',
    '라인업 효율성 점수'
]

# tabulate 출력
print(
    tabulate(
        combined_metrics_df_copy.head(10),
        headers=combined_metrics_df_copy_cols,
        tablefmt='fancy',
        showindex=False,
        colalign=("right", "right", "right", "right", "right")
    )
)

  리그 ID                        리그 이름    시즌    리그 균형도 점수    라인업 효율성 점수
---------  -------------------------------  ------  ------------------  --------------------
     5001  Pigskin Prodigal Fantasy League    2023             88.9946                84.063
     5002       Recurring Champions League    2023             94.8453                74.171
     5003                      AHAHFZZFFFL    2023             85.3159                69.888
     5004    Gridiron Gurus Fantasy League    2023             87.0678                82.268
     5005                 Best League Ever    2023             86.8592                79.642


# 샤크 리그 점수 계산
## 샤크 리그 점수 = (2 * 라인업 효율성 점수) + 리그 균형 점수

In [14]:
combined_metrics_df['shark_league_score'] = combined_metrics_df.apply(
    lambda league: (2 * league['league_juice_score']) +  league['league_balance_score'],
    axis=1
)
display(combined_metrics_df.sort_values(by='shark_league_score', ascending=False))

,league_id,league_name,year,league_balance_score,league_juice_score,shark_league_score
0,5001,Pigskin Prodigal Fantasy League,2023,88.994643,84.063173,257.120989
3,5004,Gridiron Gurus Fantasy League,2023,87.067786,82.268485,251.604756
4,5005,Best League Ever,2023,86.859170,79.642259,246.143688
1,5002,Recurring Champions League,2023,94.845345,74.171001,243.187348
2,5003,AHAHFZZFFFL,2023,85.315897,69.887841,225.091580


In [15]:
combined_metrics_df_copy = combined_metrics_df.copy()

# 천 단위 콤마 적용
combined_metrics_df_copy['league_balance_score'] = combined_metrics_df_copy['league_balance_score'].map('{:,.4f}'.format)
combined_metrics_df_copy['league_juice_score'] = combined_metrics_df_copy['league_juice_score'].map('{:,.3f}'.format)
combined_metrics_df_copy['shark_league_score'] = combined_metrics_df_copy['shark_league_score'].map('{:,.6f}'.format)

combined_metrics_df_copy_cols = [
    '리그 ID',
    '리그 이름',
    '시즌',
    '리그 균형도 점수',
    '라인업 효율성 점수',
    '샤크 리그 점수'
]

# tabulate 출력
print(
    tabulate(
        combined_metrics_df_copy.head(10),
        headers=combined_metrics_df_copy_cols,
        tablefmt='fancy',
        showindex=False,
        colalign=("right", "right", "right", "right", "right", "right")
    )
)

  리그 ID                        리그 이름    시즌    리그 균형도 점수    라인업 효율성 점수    샤크 리그 점수
---------  -------------------------------  ------  ------------------  --------------------  ----------------
     5001  Pigskin Prodigal Fantasy League    2023             88.9946                84.063           257.121
     5002       Recurring Champions League    2023             94.8453                74.171           243.187
     5003                      AHAHFZZFFFL    2023             85.3159                69.888           225.092
     5004    Gridiron Gurus Fantasy League    2023             87.0678                82.268           251.605
     5005                 Best League Ever    2023             86.8592                79.642           246.144
